In [42]:
import pandas as pd
import networkx as nx
import math
from pathlib import Path
from typing import List, Dict, Tuple, Set

# ── tweakables ───────────────────────────────────────────────────────────────
NODES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/nodes_final.csv")
EDGES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/relationships_final.csv")

BETA        = 8.0      # spreads “good” (≈0) vs “bad” (≪0) scores
LAMBDA      = 1.0      # 1.0 ⇒ cost = structural only (no embeddings)

# If you *do* want to include semantic smoothness later:
EMB_PATH    = None     # e.g. Path("node_embeddings.pt")

In [43]:
# Cell 2 ── Load graph with dynamic column detection + debug ────────────────
def load_graph(nodes_csv: Path, edges_csv: Path) -> Tuple[nx.DiGraph, Dict[int, Dict]]:
    # 1. Read CSVs
    nodes_df = pd.read_csv(nodes_csv)
    edges_df = pd.read_csv(edges_csv)
    
    # 2. Debug: show what columns we actually have
    print("→ nodes_df columns:", nodes_df.columns.tolist())
    print("→ edges_df columns:", edges_df.columns.tolist())
    
    # 3. Dynamically pick the right column names
    nid_col       = next(c for c in nodes_df.columns if "ID"   in c)
    name_col      = next(c for c in nodes_df.columns if "name" in c.lower())
    def_col       = next(c for c in nodes_df.columns if "def"  in c.lower())
    labels_col    = next(c for c in nodes_df.columns if "label" in c.lower())
    
    src_col       = next(c for c in edges_df.columns if "start" in c.lower())
    dst_col       = next(c for c in edges_df.columns if "end"   in c.lower())
    score_col     = next(c for c in edges_df.columns if "score" in c.lower())
    predicted_col = next(c for c in edges_df.columns if "predicted" in c.lower())
    
    # 4. Build the graph
    G = nx.DiGraph()
    node_props = {}
    
    for _, row in nodes_df.iterrows():
        nid = int(row[nid_col])
        props = {
            "name":       row[name_col],
            "definition": row[def_col],
            "labels":     row[labels_col].split(";")
        }
        G.add_node(nid, **props)
        node_props[nid] = props
    
    for _, row in edges_df.iterrows():
        src   = int(row[src_col])
        dst   = int(row[dst_col])
        score = float(row[score_col])
        pred  = bool(row[predicted_col])
        G.add_edge(src, dst, score=score, predicted=pred)
    
    return G, node_props

# Run it and debug
G, props = load_graph(NODES_CSV, EDGES_CSV)
print("✅ Graph loaded!")
print(f"   Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")
# show one example
u, v, d = next(iter(G.edges(data=True)))
print(f"   Example edge {u}→{v} has attrs {d}")

→ nodes_df columns: ['nodeId:ID', 'name', 'definition', 'labels:LABEL']
→ edges_df columns: [':START_ID', ':END_ID', 'type', 'score:float', 'predicted:boolean']
✅ Graph loaded!
   Nodes: 1148, Edges: 4380
   Example edge 0→8 has attrs {'score': -0.8655019998550415, 'predicted': False}


In [44]:
import networkx as nx, math, torch, numpy as np
from numpy.linalg import norm

# ---------- 1) orient every edge (prereq → dependent) ----------
H = nx.DiGraph()
for u, v, d in G.edges(data=True):
    # if reverse edge exists, keep only the prerequisite direction
    if G.has_edge(v, u) and abs(G[v][u]['score']) < abs(d['score']):
        continue                    # v→u is the stronger prereq arrow
    H.add_edge(u, v, score=d['score'])

print(f"Oriented DAG: {H.number_of_nodes():,} nodes, {H.number_of_edges():,} edges")

# ---------- 2) longest-path layer number ----------
layer = {}
for root in [n for n in H if H.in_degree(n)==0]:
    for n, depth in nx.single_source_shortest_path_length(H, root).items():
        layer[n] = max(layer.get(n, 0), depth)

Oriented DAG: 1,143 nodes, 4,380 edges


In [45]:
import torch

# if you saved with torch.save(node_embeddings, "node_embeddings.pt")
node_embeddings = torch.load("node_embeddings.pt", map_location="cpu")

# quick sanity check
print(f"Loaded {len(node_embeddings)} node embeddings")
nid, emb = next(iter(node_embeddings.items()))
print(f"Example – node {nid} → embedding shape {tuple(emb.shape)}")

Loaded 1148 node embeddings
Example – node 0 → embedding shape (128,)


In [46]:
SEM_WEIGHT   = 0.3     # semantic share (0 → ignore embeddings)
SKIP_FACTOR  = 0.3     # penalty per extra layer jump beyond +1

# pre-compute max |score|
max_abs = max(abs(d['score']) for _,_,d in H.edges(data=True))

def cosine(u, v):
    return float((u @ v) / (norm(u)*norm(v) + 1e-9))

for u, v, d in H.edges(data=True):
    struct = abs(d['score']) / max_abs

    # safe semantic term
    if SEM_WEIGHT and (u in node_embeddings) and (v in node_embeddings):
        sem = 1 - cosine(node_embeddings[u], node_embeddings[v])
    else:
        sem = 0.0

    skip = max(layer[v] - layer[u] - 1, 0) * SKIP_FACTOR

    d['cost'] = (1-SEM_WEIGHT)*struct + SEM_WEIGHT*sem + skip

In [48]:
def get_label(d):
    return d.get("name") or d.get("title")

# Build name2id, skipping label-less nodes
name2id = {}
missing_labels = []
for nid, d in H.nodes(data=True):
    lbl = get_label(d)
    if lbl is None:
        missing_labels.append(nid)
    else:
        name2id[lbl.lower()] = nid

if missing_labels:
    print(f"⚠️  skipped {len(missing_labels)} nodes with no name/title: {missing_labels[:10]}…")

# Build id2name similarly
id2name = {nid:lbl for lbl,nid in ((get_label(d), nid) for nid,d in H.nodes(data=True)) if lbl}

⚠️  skipped 1143 nodes with no name/title: [0, 8, 9, 43, 63, 76, 1, 2, 4, 14]…


In [50]:
from heapq import heappush, heappop

# ─── 1. Build label maps from props, not H ─────────────────────────────────
def get_label(d):
    # for skills/concepts: "name"; for jobs: "title"
    return d.get("name") or d.get("title")

name2id = {}
for nid, d in props.items():
    lbl = get_label(d)
    if lbl:
        name2id[lbl.lower()] = nid
    else:
        print(f"⚠️  Node {nid!r} has no name or title in props")

id2name = {nid: get_label(d) for nid, d in props.items() if get_label(d)}

# ─── 2. Dijkstra to find best cost path to one skill ────────────────────────
def best_path_to_skill(skill_id):
    roots = [n for n in H if H.in_degree(n)==0]
    dist  = {n: 0.0 for n in roots}
    prev  = {}
    pq    = [(0.0, n) for n in roots]

    while pq:
        cost, u = heappop(pq)
        if u == skill_id:
            # reconstruct path
            path = []
            while u in prev:
                path.append(u)
                u = prev[u]
            path.append(u)
            return cost, list(reversed(path))
        for _, v, d in H.out_edges(u, data=True):
            alt = cost + d["cost"]
            if alt < dist.get(v, float("inf")):
                dist[v] = alt
                prev[v] = u
                heappush(pq, (alt, v))
    return None, None  # unreachable

# ─── 3. Report for each required skill of the job ──────────────────────────
def report_job(job_key):
    jid = name2id.get(job_key.lower())
    if jid is None:
        raise KeyError(f"Job '{job_key}' not found in props")

    # gather direct requires (in-edges or out-edges fallback)
    req = {src for src,_ in G.in_edges(jid)} or \
          {dst for _,dst in G.out_edges(jid)}

    print(f"\n=== {job_key.upper()} requires {len(req)} skills ===\n")
    for sid in sorted(req, key=lambda x: id2name.get(x, "").lower()):
        cost, path = best_path_to_skill(sid)
        skill = id2name.get(sid, f"<{sid}>")
        if path is None:
            print(f"✘ {skill:30} — no prereq path in H\n")
            continue

        names = " → ".join(id2name[n] for n in path)
        if len(path) == 1:
            print(f"• {skill:30}  (no prerequisites)")
        else:
            print(f"• {skill:30}  steps={len(path)-1} cost={cost:.3f}")
            print("    ", names)
        print()

# ─── Example run ───────────────────────────────────────────────────────────
report_job("machine learning engineer")


=== MACHINE LEARNING ENGINEER requires 30 skills ===

• algorithms                      steps=2 cost=0.013
     genai manager → genai → algorithms

• aws                             steps=2 cost=0.045
     machine learning developer → sagemaker → aws

• c++                             steps=2 cost=0.398
     llm intern → address locator → c++

• collaboration                   steps=1 cost=0.000
     ai architect → collaboration

• communications                  steps=1 cost=0.000
     ai architect → communications

• computer science                steps=6 cost=0.113
     computer animation → rendering (computer graphics) → computer graphics → image representation → feature extraction → algorithm → computer science

• creativity                      steps=1 cost=0.000
     ai research researcher → creativity

• data science                    steps=2 cost=0.033
     machine learning engineer → docker → data science

• deep learning                   steps=2 cost=0.146
     llm devel

In [52]:
# ---------------------------------------------------------------------------
#  CONFIG  – edit these two lines only
# ---------------------------------------------------------------------------
# ── tweakables ───────────────────────────────────────────────────────────────
NODES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/nodes_final.csv")
EDGES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/relationships_final.csv")
JOB_TITLE = "machine learning engineer"
# ---------------------------------------------------------------------------

import pandas as pd, networkx as nx, math, heapq

########################################################################
# 1.  Load full graph
########################################################################
G_full = nx.DiGraph()
nodes = pd.read_csv(NODES_CSV)
edges = pd.read_csv(EDGES_CSV)

for _, r in nodes.iterrows():
    G_full.add_node(
        int(r["nodeId:ID"]),
        label_list = r["labels:LABEL"].split(";"),
        title      = r.get("title", None),        # may be NaN
        name       = r.get("name", None)
    )

for _, r in edges.iterrows():
    G_full.add_edge(
        int(r[":START_ID"]),
        int(r[":END_ID"]),
        score     = float(r["score:float"]),
        predicted = bool(r["predicted:boolean"])
    )

########################################################################
# 2.  Keep only Concept / HardSkill / Technology nodes, drop Jobs
########################################################################
ALLOWED = {"Concept", "HardSkill", "Technology"}

sub_nodes = [
    n for n,d in G_full.nodes(data=True)
    if any(l in ALLOWED for l in d["label_list"])
]
G = G_full.subgraph(sub_nodes).copy()

########################################################################
# 3.  Reverse edges so they flow prerequisite → dependent
########################################################################
H = G.reverse(copy=True)

########################################################################
# 4.  Assign edge cost = |score| + 0.5·predicted
########################################################################
for u, v, d in H.edges(data=True):
    d["cost"] = abs(G_full[v][u]["score"]) + (0.5 if G_full[v][u]["predicted"] else 0.0)

########################################################################
# 5.  Build label maps from original node data
########################################################################
def label(nid):
    d = G_full.nodes[nid]
    return d["name"] or d["title"] or f"<{nid}>"

name2id = {label(n).lower(): n for n in G_full}
id2name = {n: label(n) for n in G_full}

########################################################################
# 6.  Find the Job node and its required skills
########################################################################
job_id = name2id.get(JOB_TITLE.lower())
if job_id is None:
    raise KeyError(f"Job '{JOB_TITLE}' not found")

required = {dst for _, dst in G_full.out_edges(job_id)}          # Job → Skill

########################################################################
# 7.  Roots (nodes with no incoming prereq edges)
########################################################################
roots = [n for n in H if H.in_degree(n) == 0]

########################################################################
# 8.  DP: best predecessor + cost for every node
########################################################################
best_cost = {n: 0.0 for n in roots}
parent    = {}

for n in nx.topological_sort(H):
    for _, child, d in H.out_edges(n, data=True):
        alt = best_cost[n] + d["cost"]
        if alt < best_cost.get(child, math.inf):
            best_cost[child] = alt
            parent[child]    = n

########################################################################
# 9.  Function to back-track a path
########################################################################
def path_to(node):
    if node not in best_cost:                  # unreachable
        return None
    path = [node]
    while node in parent:
        node = parent[node]
        path.append(node)
    return list(reversed(path))

########################################################################
# 10.  Print one best path (concepts + hard skills only) per required skill
########################################################################
print(f"\n=====  {JOB_TITLE.upper()}  – {len(required)} required skills  =====\n")
for sid in sorted(required, key=lambda x: id2name[x].lower()):
    if sid not in H:                           # e.g. SoftSkill → skip
        print(f"• {id2name[sid]:30} (soft or missing – skipped)\n")
        continue

    p = path_to(sid)
    if p is None:
        print(f"✘ {id2name[sid]:30}  — no prerequisite chain in data\n")
        continue

    names = " → ".join(id2name[n] for n in p)
    if len(p) == 1:
        print(f"• {id2name[sid]:30}  (no prerequisites)\n")
    else:
        print(f"• {id2name[sid]:30}  steps={len(p)-1}  cost={best_cost[sid]:.3f}")
        print("    ", names, "\n")


=====  MACHINE LEARNING ENGINEER  – 30 required skills  =====

• algorithms                      (no prerequisites)

• aws                             (no prerequisites)

• c++                             (no prerequisites)

• collaboration                  (soft or missing – skipped)

• communications                 (soft or missing – skipped)

• computer science                steps=2  cost=0.016
     arithmetic → computer → computer science 

• creativity                     (soft or missing – skipped)

• data science                    (no prerequisites)

• deep learning                   (no prerequisites)

• docker                          steps=1  cost=0.678
     data science → docker 

• gcp                             (no prerequisites)

• google cloud platform (gcp)     steps=1  cost=0.577
     gcp → google cloud platform (gcp) 

• infrastructure                 (soft or missing – skipped)

• innovation                     (soft or missing – skipped)

• integration         

In [12]:
# ───────────────────────────── Imports & config ────────────────────────────
import pandas as pd, networkx as nx, torch
from pathlib import Path
from numpy.linalg import norm

NODES_CSV = Path("/teamspace/studios/this_studio/neo4j_export_csv/nodes_final.csv")
EDGES_CSV = Path("/teamspace/studios/this_studio/neo4j_export_csv/relationships_final.csv")
EMB_FILE  = Path("node_embeddings.pt")

JOB_TITLE   = "mlops engineer"        # ← change job here
DELTA       = 1.1                   # reward per hop  (>1)
ALPHA       = 0.6                   # weight on |score|
BETA        = 0.4                   # weight on semantic penalty
PRED_PENALTY= 0.5                   # add to |score| if edge.predicted=true
END_LABELS  = {"HardSkill","Technology"}
SUGGEST_K   = 5                  # max Concepts to suggest
# ────────────────────────────────────────────────────────────────────────────

# 1) LOAD
nodes_df  = pd.read_csv(NODES_CSV)
edges_df  = pd.read_csv(EDGES_CSV)
emb       = torch.load(str(EMB_FILE), map_location="cpu")

label_of = {int(r["nodeId:ID"]): set(r["labels:LABEL"].split(";"))
            for _,r in nodes_df.iterrows()}
def name_of(nid):
    row = nodes_df.loc[nodes_df["nodeId:ID"]==nid].iloc[0]
    return row.get("name") or row.get("title") or str(nid)
id2name = {n: name_of(n) for n in label_of}
name2id = {v.lower(): k for k,v in id2name.items()}

job_id = name2id[JOB_TITLE.lower()]
Smax   = edges_df["score:float"].abs().max()

def cosine(a,b): return float((a@b)/(norm(a)*norm(b)+1e-9))

# 2) DAG H = Concept → dependent, with Technology never as dependent
H = nx.DiGraph(); H.add_nodes_from(label_of)
for _, r in edges_df.iterrows():
    dep, prereq = int(r[":START_ID"]), int(r[":END_ID"])   # CSV direction
    if "Concept" not in label_of.get(prereq, ()):          # source must be Concept
        continue
    if "Technology" in label_of.get(dep, ()):              # Technology never inside
        continue
    raw = abs(float(r["score:float"]))
    cost = raw + (PRED_PENALTY if r["predicted:boolean"] else 0.0)
    if nx.has_path(H, dep, prereq):                        # avoid cycles
        continue
    if H.has_edge(prereq, dep):
        H[prereq][dep]["cost"] = min(H[prereq][dep]["cost"], cost)
        H[prereq][dep]["raw"]  = min(H[prereq][dep]["raw"],  raw)
    else:
        H.add_edge(prereq, dep, cost=cost, raw=raw)

assert nx.is_directed_acyclic_graph(H)

# 3) DP   (deepest, best-cost Concept chain)
dp, prev = {n:-1e9 for n in H}, {}
for root in [n for n in H if H.in_degree(n)==0 and "Concept" in label_of[n]]:
    dp[root] = 0.0

for u in nx.topological_sort(H):
    if dp[u] < -1e8: continue
    for _, v, d in H.out_edges(u, data=True):
        s_norm  = d["raw"]/Smax if Smax else 0
        sem_pen = 1 - cosine(emb[u],emb[v]) if (u in emb and v in emb) else 1
        w       = DELTA - ALPHA*s_norm - BETA*sem_pen
        if dp[u]+w > dp[v]:
            dp[v], prev[v] = dp[u]+w, u

def backtrack(n):
    if dp.get(n,-1e9)<-1e8: return None
    chain=[n]
    while chain[-1] in prev: chain.append(prev[chain[-1]])
    return list(reversed(chain))

# helper: up-to-K Concept parents (never Technology)
def suggest_parents(skill):
    inc = [p for p,_,_ in H.in_edges(skill) if "Concept" in label_of[p]]
    if inc:
        inc.sort(key=lambda p: H[p][skill]["cost"])
        return inc[:SUGGEST_K]
    # fallback: Concept→skill edges in raw CSV that were filtered
    rows = edges_df.loc[edges_df[":END_ID"]==skill, ":START_ID"].astype(int)
    cand = [n for n in rows if "Concept" in label_of.get(n,())]
    return cand[:SUGGEST_K]

# 4) REPORT
reqs = set(edges_df.loc[edges_df[":START_ID"]==job_id, ":END_ID"].astype(int))
print(f"\n{JOB_TITLE.upper()} – {len(reqs)} required skills\n")

for sid in sorted(reqs, key=lambda x:id2name[x].lower()):
    if not (label_of[sid] & END_LABELS): continue

    chain = backtrack(sid)
    if chain and len(chain)>1:
        print(f"• {id2name[sid]:30}  depth={len(chain)-1}")
        print("    ", " → ".join(id2name[n] for n in chain), "\n")
        continue

    # unreachable: show suggestions (Concepts only)
    print(f"✘ {id2name[sid]:30} — no Concept prereqs in graph")
    for p in suggest_parents(sid):
        alt = backtrack(p) or [p]
        print("     •", " → ".join(id2name[n] for n in alt))
    print()


MLOPS ENGINEER – 29 required skills

✘ automation                     — no Concept prereqs in graph

✘ aws                            — no Concept prereqs in graph
     • foundations of mathematics → mathematics → probability → conditional probability → bayes theorem → graph theory → computer science → computability → algorithm → algorithm design → computational complexity theory → computer programming → operating system → interprocess communication → message passing interface
     • amazon lex
     • sagemaker

✘ azure                          — no Concept prereqs in graph
     • foundations of mathematics → mathematics → probability → descriptive statistics and hypothesis testing → maximum likelihood estimation
     • ai tools
     • jira
     • llms
     • microsoft azure

• computer science                depth=6
     foundations of mathematics → mathematics → probability → conditional probability → bayes theorem → graph theory → computer science 

✘ containerization              

In [17]:
# ─── 0. Safe name_of (if you haven’t already) ────────────────────────────
def name_of(nid):
    row = nodes_df.loc[nodes_df["nodeId:ID"] == nid].iloc[0]
    if "name" in row and pd.notna(row["name"]):
        return row["name"]
    elif "title" in row and pd.notna(row["title"]):
        return row["title"]
    else:
        return str(nid)

# ─── 1. Build id↔name maps ────────────────────────────────────────────────
id2name = { int(r["nodeId:ID"]) : name_of(int(r["nodeId:ID"])) 
            for _,r in nodes_df.iterrows() }
name2id = { v.lower(): k for k,v in id2name.items() }

# ─── 2. Lookup “Computer Science” ─────────────────────────────────────────
cs_id = name2id.get("machine learning")
if cs_id is None:
    raise KeyError("Could not find a node called 'computer science' in name2id")

print(f"Node ID for Computer Science: {cs_id}")
print(f"Labels on this node: {nodes_df.loc[nodes_df['nodeId:ID']==cs_id, 'labels:LABEL'].iloc[0]}\n")

# ─── 3. Show 5 incoming REQUIRES edges ────────────────────────────────────
print("→ First 5 incoming edges (dependent → computer science):")
inc = edges_df[edges_df[":END_ID"] == cs_id].head(5)
for _, row in inc.iterrows():
    src = int(row[":START_ID"])
    print(f"   • {id2name[src]} ─REQUIRES({row['score:float']:.3f})→ {id2name[cs_id]}")

# ─── 4. Show 5 outgoing REQUIRES edges ────────────────────────────────────
print("\n→ First 5 outgoing edges (computer science → prerequisite):")
out = edges_df[edges_df[":START_ID"] == cs_id].head(5)
for _, row in out.iterrows():
    tgt = int(row[":END_ID"])
    print(f"   • {id2name[cs_id]} ─REQUIRES({row['score:float']:.3f})→ {id2name[tgt]}")

Node ID for Computer Science: 285
Labels on this node: Concept;HardSkill

→ First 5 incoming edges (dependent → computer science):
   • reinforcement learning ─REQUIRES(-0.135)→ machine learning
   • unsupervised learning ─REQUIRES(-0.286)→ machine learning
   • document classification ─REQUIRES(-0.521)→ machine learning
   • natural language processing ─REQUIRES(-0.318)→ machine learning
   • speech recognition ─REQUIRES(-0.032)→ machine learning

→ First 5 outgoing edges (computer science → prerequisite):
   • machine learning ─REQUIRES(-0.001)→ probability
   • machine learning ─REQUIRES(-0.178)→ artificial intelligence
   • machine learning ─REQUIRES(-0.083)→ algorithm design
   • machine learning ─REQUIRES(-0.036)→ graph theory
   • machine learning ─REQUIRES(-0.020)→ computer programming


In [20]:
# ───────────────────────────── PATH‐FINDING CELL ─────────────────────────────
import pandas as pd, networkx as nx, torch, math
from pathlib import Path
from numpy.linalg import norm

# ─── config ────────────────────────────────────────────────
NODES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/nodes_final.csv")
EDGES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/relationships_final.csv")
EMB_FILE    = Path("node_embeddings.pt")

JOB         = "mlops engineer"                # ← your job title
CONCEPT     = "Concept"
HARDSKILL   = "HardSkill"
TECH        = "Technology"
END_LABELS  = {HARDSKILL, TECH}              # only these count as end‐skills
SUGGEST_K   = 3                              # how many parents to suggest

Δ, α, β, PRED = 1.1, 0.6, 0.4, 0.5            # scoring constants
# ────────────────────────────────────────────────────────────────────────────

# 1) LOAD
nodes = pd.read_csv(NODES_CSV)
edges = pd.read_csv(EDGES_CSV)
emb   = torch.load(str(EMB_FILE), map_location="cpu")

label_of = {
    int(r["nodeId:ID"]): set(r["labels:LABEL"].split(";"))
    for _, r in nodes.iterrows()
}

def name_of(n):
    row = nodes.loc[nodes["nodeId:ID"]==n].iloc[0]
    return row.get("name") or row.get("title") or str(n)

id2name = {n: name_of(n) for n in label_of}
name2id = {v.lower(): k for k,v in id2name.items()}

job_id = name2id[JOB.lower()]
Smax   = edges["score:float"].abs().max()
cosine = lambda a,b: float((a@b)/(norm(a)*norm(b)+1e-9))

# 2) Build learning‐DAG H: prereq → dependent,
#    allow prereq if it’s Concept or HardSkill, but never if Technology.
H = nx.DiGraph(); H.add_nodes_from(label_of)
for _, r in edges.iterrows():
    dep, pre = int(r[":START_ID"]), int(r[":END_ID"])  # CSV orientation
    labs_pre = label_of.get(pre, ())
    labs_dep = label_of.get(dep, ())

    # prereq must be Concept or HardSkill; dependent must not be Technology
    if not (CONCEPT in labs_pre or HARDSKILL in labs_pre):
        continue
    if TECH in labs_dep:
        continue

    raw  = abs(float(r["score:float"]))
    cost = raw + (PRED if r["predicted:boolean"] else 0.0)

    # avoid cycles before adding
    if nx.has_path(H, dep, pre):
        continue

    # insert / tighten
    if H.has_edge(pre, dep):
        H[pre][dep]["cost"] = min(H[pre][dep]["cost"], cost)
        H[pre][dep]["raw"]  = min(H[pre][dep]["raw"],  raw)
    else:
        H.add_edge(pre, dep, cost=cost, raw=raw)

assert nx.is_directed_acyclic_graph(H), "H must be a DAG!"

# 3) DP: deepest + best‐score chain to every node
dp, prev = {n:-math.inf for n in H}, {}
# seeds: any Concept or HardSkill with zero prereqs
for n in H:
    if H.in_degree(n)==0 and (CONCEPT in label_of[n] or HARDSKILL in label_of[n]):
        dp[n] = 0.0

for u in nx.topological_sort(H):
    if dp[u] < -1e8:
        continue
    for _, v, d in H.out_edges(u, data=True):
        s_norm  = d["raw"] / Smax if Smax else 0.0
        sem_pen = 1 - cosine(emb[u], emb[v]) if (u in emb and v in emb) else 1.0
        gain    = Δ - α*s_norm - β*sem_pen
        cand    = dp[u] + gain
        if cand > dp[v]:
            dp[v], prev[v] = cand, u

def build_chain(n):
    if dp.get(n, -math.inf) < -1e8:
        return None
    path = [n]
    while path[-1] in prev:
        path.append(prev[path[-1]])
    return list(reversed(path))

# 4) Suggestions for orphan skills
def suggest_parents(skill):
    # raw CSV: skill → prereq; we want only Concept/HardSkill prereqs
    rows = edges.loc[edges[":START_ID"] == skill, ":END_ID"].astype(int)
    cands = [p for p in rows if (CONCEPT in label_of.get(p,()) or HARDSKILL in label_of.get(p,()))]
    # rank by raw cost
    scored = []
    for p in cands:
        r = edges[(edges[":START_ID"]==skill)&(edges[":END_ID"]==p)].iloc[0]
        sc = abs(float(r["score:float"])) + (PRED if r["predicted:boolean"] else 0.0)
        scored.append((p, sc))
    scored.sort(key=lambda x: x[1])
    return [p for p,_ in scored[:SUGGEST_K]]

# 5) REPORT for required skills
reqs = set(edges.loc[edges[":START_ID"]==job_id, ":END_ID"].astype(int))
print(f"\n{JOB.upper()} — {len(reqs)} required skills\n")

for sid in sorted(reqs, key=lambda x:id2name[x].lower()):
    # only HardSkill/Technology as end targets
    if not (label_of[sid] & END_LABELS):
        continue

    chain = build_chain(sid)
    if chain and len(chain) > 1:
        print(f"• {id2name[sid]:30}  depth={len(chain)-1}")
        print("    ", " → ".join(id2name[n] for n in chain), "\n")
    else:
        print(f"✘ {id2name[sid]:30}  — no prereq chain in graph")
        for p in suggest_parents(sid):
            alt = build_chain(p) or [p]
            print("    •", " → ".join(id2name[n] for n in alt))
        print()


MLOPS ENGINEER — 29 required skills

✘ automation                      — no prereq chain in graph

✘ aws                             — no prereq chain in graph

✘ azure                           — no prereq chain in graph

• computer science                depth=6
     foundations of mathematics → mathematics → probability → conditional probability → bayes theorem → graph theory → computer science 

✘ containerization                — no prereq chain in graph

✘ data science                    — no prereq chain in graph

✘ devops                          — no prereq chain in graph

✘ docker                          — no prereq chain in graph
    • data science

✘ gcp                             — no prereq chain in graph

• google cloud platform (gcp)     depth=1
     gcp → google cloud platform (gcp) 

✘ kubernetes                      — no prereq chain in graph

• machine learning                depth=22
     foundations of mathematics → mathematics → probability → conditional proba

In [20]:
# ─────────────────────────────── PATH-FINDING CELL ──────────────────────────────
import pandas as pd
import networkx as nx
import torch
import math
from pathlib import Path
from numpy.linalg import norm

# ─── config ────────────────────────────────────────────────────────────────────
NODES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/nodes_final.csv")
EDGES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/relationships_final.csv")
EMB_FILE    = Path("node_embeddings.pt")

JOB         = "computer vision engineer"                # ← your job title
CONCEPT     = "Concept"
HARDSKILL   = "HardSkill"
TECH        = "Technology"
END_LABELS  = {HARDSKILL, TECH}              # only these count as end-skills
SUGGEST_K   = 3                              # how many concepts to suggest for orphans

Δ, α, β, PRED = 0.3, 0.9, 0.1, 0.7        # scoring constants
# ─────────────────────────────────────────────────────────────────────────────

# 1) LOAD CSVs & EMBEDDINGS
nodes = pd.read_csv(NODES_CSV)
edges = pd.read_csv(EDGES_CSV)
emb   = torch.load(str(EMB_FILE), map_location="cpu")

# build label lookup
label_of = {
    int(r["nodeId:ID"]): set(r["labels:LABEL"].split(";"))
    for _, r in nodes.iterrows()
}

# safe name lookup
def name_of(n):
    row = nodes.loc[nodes["nodeId:ID"] == n].iloc[0]
    if "name" in row and pd.notna(row["name"]):
        return row["name"]
    if "title" in row and pd.notna(row["title"]):
        return row["title"]
    return str(n)

# id↔name maps
id2name = {nid: name_of(nid) for nid in label_of}
name2id = {n.lower(): nid for nid, n in id2name.items()}

job_id = name2id.get(JOB.lower())
if job_id is None:
    raise KeyError(f"Job '{JOB}' not found in nodes")
Smax = edges["score:float"].abs().max()

# cosine similarity
def cosine(u, v):
    return float((u @ v) / (norm(u) * norm(v) + 1e-9))

# 2) BUILD learning-DAG H: prereq → dependent
H = nx.DiGraph()
H.add_nodes_from(label_of.keys())

for _, r in edges.iterrows():
    dep, pre = int(r[":START_ID"]), int(r[":END_ID"])  # CSV: dependent→prereq
    labs_pre = label_of.get(pre, ())
    labs_dep = label_of.get(dep, ())

    # only keep if prereq is Concept or HardSkill, and dependent isn't Technology
    if not (CONCEPT in labs_pre or HARDSKILL in labs_pre):
        continue
    if TECH in labs_dep:
        continue

    raw  = abs(float(r["score:float"]))
    cost = raw + (PRED if r["predicted:boolean"] else 0.0)

    # avoid cycles
    if nx.has_path(H, dep, pre):
        continue

    # insert or tighten existing edge
    if H.has_edge(pre, dep):
        H[pre][dep]["cost"] = min(H[pre][dep]["cost"], cost)
        H[pre][dep]["raw"]  = min(H[pre][dep]["raw"],  raw)
    else:
        H.add_edge(pre, dep, cost=cost, raw=raw)

assert nx.is_directed_acyclic_graph(H), "H must be a DAG!"

# 3) DP: longest + best-score path to every node
dp   = {n: -math.inf for n in H}
prev = {}

# seed roots: any Concept/HardSkill with zero prerequisites
for n in H:
    if H.in_degree(n) == 0 and (CONCEPT in label_of[n] or HARDSKILL in label_of[n]):
        dp[n] = 0.0

for u in nx.topological_sort(H):
    if dp[u] < -1e8:
        continue
    for _, v, d in H.out_edges(u, data=True):
        s_norm  = d["raw"] / Smax if Smax else 0.0
        sem_pen = (1 - cosine(emb[u], emb[v])) if (u in emb and v in emb) else 1.0
        gain    = Δ - α * s_norm - β * sem_pen
        cand    = dp[u] + gain
        if cand > dp[v]:
            dp[v], prev[v] = cand, u

# back-track chain builder
def build_chain(n):
    if dp.get(n, -math.inf) < -1e8:
        return None
    path = [n]
    while path[-1] in prev:
        path.append(prev[path[-1]])
    return list(reversed(path))

# ─────────── USER INPUT ──────────────────────────────────
# ---------------- USER INPUT -----------------
user_skills = ["graph theory", "computer vision","neural network","reinforcement learning"]          # ← whatever they enter
user_ids    = { name2id[s.lower()] for s in user_skills if s.lower() in name2id }
# ---------------------------------------------

def make_filtered_graph(target_id):
    """Subgraph of H with cosine≥threshold *plus* all roots *plus* user skills."""
    target_emb = emb.get(target_id)
    roots = {n for n in H if H.in_degree(n)==0
                          and (CONCEPT in label_of[n] or HARDSKILL in label_of[n])}

    good = set(roots) | {target_id} | user_ids        # <- keep user skills!

    if target_emb is not None:
        for n in H:
            if n in emb and (CONCEPT in label_of[n] or HARDSKILL in label_of[n]):
                if cosine(emb[n], target_emb) >= DOMAIN_THRESH:
                    good.add(n)

    return H.subgraph(good).copy()

# helper: trim a path so it starts with the first node the user already knows
def trim_to_user(path):
    for i, n in enumerate(path):
        if n in user_ids:
            return path[i:]
    return None          # no user skill inside

# ─────────── 5. REPORT (replaces your current loop) ──────
reqs = set(edges.loc[edges[":START_ID"] == job_id, ":END_ID"].astype(int))
print(f"\n{JOB.upper()} — {len(reqs)} required skills\n")

for sid in sorted(reqs, key=lambda x: id2name[x].lower()):
    if not (label_of[sid] & END_LABELS):
        continue  # ignore SoftSkills, Jobs, etc.

    chain = build_chain(sid)

    # 1) If chain exists, try to anchor it at a user skill
    if chain and len(chain) > 1:
        anchored = trim_to_user(chain) if user_ids else None
        final_chain = anchored if anchored else chain

        if len(final_chain) == 1:          # skill already known
            print(f"• {id2name[sid]:30}  (already known)\n")
            continue

        print(f"• {id2name[sid]:30}  depth={len(final_chain)-1}")
        print("    ", " → ".join(id2name[n] for n in final_chain), "\n")
        continue

    # 2) Orphan skill: show dependents to learn next
    print(f"✘ {id2name[sid]:30}  — no prereq chain in graph")
    deps = edges.loc[edges[":END_ID"] == sid, ":START_ID"].astype(int)
    cands = [d for d in deps if (CONCEPT in label_of.get(d,()) or HARDSKILL in label_of.get(d,()))]

    scored = []
    for d in cands:
        r = edges[(edges[":START_ID"]==d)&(edges[":END_ID"]==sid)].iloc[0]
        sc = abs(float(r["score:float"])) + (PRED if r["predicted:boolean"] else 0.0)
        scored.append((d, sc))
    scored.sort(key=lambda x: x[1])

    print("    Concepts that depend on this skill (learn next):")
    for d, _ in scored[:SUGGEST_K]:
        alt = build_chain(d) or [d]
        # anchor each alt chain at user skill if possible
        anchored = trim_to_user(alt) if user_ids else None
        final_alt = anchored if anchored else alt
        print("    •", " → ".join(id2name[n] for n in final_alt))
    print()


COMPUTER VISION ENGINEER — 29 required skills

✘ algorithms                      — no prereq chain in graph
    Concepts that depend on this skill (learn next):
    • genai
    • llms
    • algorithms → address locator

✘ aws                             — no prereq chain in graph
    Concepts that depend on this skill (learn next):
    • sagemaker
    • amazon lex
    • graph theory → computer science → computability → algorithm → algorithm design → computational complexity theory → computer programming → operating system → interprocess communication → message passing interface

✘ c++                             — no prereq chain in graph
    Concepts that depend on this skill (learn next):
    • graph theory → computer science → computability → algorithm → algorithm design → computational complexity theory → computer programming → operating system → interprocess communication → message passing interface
    • graph theory → computer science → computability → algorithm → p versus np p

In [33]:
# ─────────────────────────────── PATH-FINDING CELL ──────────────────────────────
import pandas as pd
import networkx as nx
import torch
import math
from pathlib import Path
from numpy.linalg import norm
import heapq
from collections import Counter

# ─── config ────────────────────────────────────────────────────────────────────
NODES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/nodes_final.csv")
EDGES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/relationships_final.csv")
EMB_FILE    = Path("node_embeddings.pt")

JOB         = "ai architect"                # ← your job title
CONCEPT     = "Concept"
HARDSKILL   = "HardSkill"
TECH        = "Technology"
END_LABELS  = {HARDSKILL, TECH}              # only these count as end-skills
SUGGEST_K   = 5                              # how many concepts to suggest for orphans
MAX_PATH_LENGTH = 15                     # maximum reasonable path length

# Scoring parameters
Δ, α, β, PRED = 0.3, 0.9, 0.1, 0.7        # scoring constants
DOMAIN_THRESH = 0.5                        # semantic similarity threshold
SIM_BOOST = 0.6                         # boost for semantically similar nodes
LENGTH_PENALTY = 0.05                  # penalty for longer paths
ORPHAN_SIM_THRESHOLD = 0.3            # threshold for connecting orphan skills to user skills
# ─────────────────────────────────────────────────────────────────────────────

# 1) LOAD CSVs & EMBEDDINGS
nodes = pd.read_csv(NODES_CSV)
edges = pd.read_csv(EDGES_CSV)
emb   = torch.load(str(EMB_FILE), map_location="cpu")

# build label lookup
label_of = {
    int(r["nodeId:ID"]): set(r["labels:LABEL"].split(";"))
    for _, r in nodes.iterrows()
}

# safe name lookup
def name_of(n):
    row = nodes.loc[nodes["nodeId:ID"] == n].iloc[0]
    if "name" in row and pd.notna(row["name"]):
        return row["name"]
    if "title" in row and pd.notna(row["title"]):
        return row["title"]
    return str(n)

# id↔name maps
id2name = {nid: name_of(nid) for nid in label_of}
name2id = {n.lower(): nid for nid, n in id2name.items()}

job_id = name2id.get(JOB.lower())
if job_id is None:
    raise KeyError(f"Job '{JOB}' not found in nodes")
Smax = edges["score:float"].abs().max()

# cosine similarity
def cosine(u, v):
    return float((u @ v) / (norm(u) * norm(v) + 1e-9))

# 2) BUILD learning-DAG H: prereq → dependent
H = nx.DiGraph()
H.add_nodes_from(label_of.keys())

# Extract all edge information first to help identify important relationships
edge_counts = Counter()
for _, r in edges.iterrows():
    dep, pre = int(r[":START_ID"]), int(r[":END_ID"])  # CSV: dependent→prereq
    edge_counts[(dep, pre)] += 1

for _, r in edges.iterrows():
    dep, pre = int(r[":START_ID"]), int(r[":END_ID"])  # CSV: dependent→prereq
    labs_pre = label_of.get(pre, ())
    labs_dep = label_of.get(dep, ())

    # only keep if prereq is Concept or HardSkill, and dependent isn't Technology
    if not (CONCEPT in labs_pre or HARDSKILL in labs_pre):
        continue
    if TECH in labs_dep:
        continue

    raw  = abs(float(r["score:float"]))
    
    # Give small boost to frequently occurring edges - indicates importance
    importance_factor = min(1.0 + 0.05 * edge_counts[(dep, pre)], 1.3)
    raw = raw * importance_factor
    
    cost = raw + (PRED if r["predicted:boolean"] else 0.0)

    # Semantic similarity boost
    if pre in emb and dep in emb:
        sim = cosine(emb[pre], emb[dep])
        if sim > 0.7:  # High similarity
            cost = cost * (1.0 - SIM_BOOST)  # Lower cost for similar concepts

    # avoid cycles
    if nx.has_path(H, dep, pre):
        continue

    # insert or tighten existing edge
    if H.has_edge(pre, dep):
        H[pre][dep]["cost"] = min(H[pre][dep]["cost"], cost)
        H[pre][dep]["raw"]  = min(H[pre][dep]["raw"],  raw)
    else:
        H.add_edge(pre, dep, cost=cost, raw=raw)

assert nx.is_directed_acyclic_graph(H), "H must be a DAG!"

# Find structural gaps in the knowledge graph
def detect_and_fill_gaps():
    """
    Detect structural gaps in the graph by finding skills that should be connected
    based on semantic similarity but aren't linked in the graph.
    """
    tech_nodes = [n for n in H.nodes() if TECH in label_of.get(n, set())]
    skills_nodes = [n for n in H.nodes() if HARDSKILL in label_of.get(n, set())]
    concept_nodes = [n for n in H.nodes() if CONCEPT in label_of.get(n, set())]
    
    additions = 0
    
    # Look for technologies that should connect to relevant skills
    for tech in tech_nodes:
        if tech not in emb:
            continue
            
        # Find semantically similar skills or concepts
        similar_nodes = []
        for skill in skills_nodes + concept_nodes:
            if skill not in emb:
                continue
                
            sim = cosine(emb[tech], emb[skill])
            if sim > 0.65:  # High similarity threshold
                similar_nodes.append((skill, sim))
        
        # Sort by similarity and connect the top matches if not already connected
        similar_nodes.sort(key=lambda x: x[1], reverse=True)
        for skill, sim in similar_nodes[:3]:  # Top 3 most similar
            if not H.has_edge(skill, tech) and not nx.has_path(H, tech, skill):
                # Add edge from skill to tech (skill is prerequisite for tech)
                H.add_edge(skill, tech, cost=0.6, raw=0.6)
                additions += 1
    
    # Connect isolated skills to relevant concepts
    orphan_skills = [n for n in H.nodes() 
                    if HARDSKILL in label_of.get(n, set()) 
                    and H.in_degree(n) == 0
                    and H.out_degree(n) == 0]
    
    for orphan in orphan_skills:
        if orphan not in emb:
            continue
            
        # Find relevant concepts
        similar_concepts = []
        for concept in concept_nodes:
            if concept not in emb:
                continue
                
            sim = cosine(emb[orphan], emb[concept])
            if sim > 0.6:
                similar_concepts.append((concept, sim))
        
        # Connect to top matches
        similar_concepts.sort(key=lambda x: x[1], reverse=True)
        for concept, sim in similar_concepts[:2]:  # Top 2
            # Add bidirectional edges to represent soft relationships
            if not nx.has_path(H, orphan, concept) and not nx.has_path(H, concept, orphan):
                H.add_edge(concept, orphan, cost=0.7, raw=0.7)
                additions += 1
    
    print(f"Added {additions} edges to fill structural gaps")
    return additions

# Fill graph gaps
detect_and_fill_gaps()

# 3) DP: longest + best-score path to every node with improved path quality
dp   = {n: -math.inf for n in H}
prev = {}
path_lens = {n: 0 for n in H}  # Keep track of path lengths

# seed roots: any Concept/HardSkill with zero prerequisites
for n in H:
    if H.in_degree(n) == 0 and (CONCEPT in label_of[n] or HARDSKILL in label_of[n]):
        dp[n] = 0.0
        path_lens[n] = 1

# Improved DP prioritizing path quality and reasonable length
for u in nx.topological_sort(H):
    if dp[u] < -1e8:
        continue
    for _, v, d in H.out_edges(u, data=True):
        s_norm  = d["raw"] / Smax if Smax else 0.0
        sem_pen = (1 - cosine(emb[u], emb[v])) if (u in emb and v in emb) else 1.0
        
        # Add length penalty to discourage excessively long paths
        length_factor = LENGTH_PENALTY * path_lens[u]
        
        gain = Δ - α * s_norm - β * sem_pen - length_factor
        cand = dp[u] + gain
        
        # Only consider if path length is reasonable
        new_path_len = path_lens[u] + 1
        if new_path_len <= MAX_PATH_LENGTH and cand > dp[v]:
            dp[v], prev[v] = cand, u
            path_lens[v] = new_path_len

# back-track chain builder with length consideration
def build_chain(n):
    if dp.get(n, -math.inf) < -1e8:
        return None
    path = [n]
    while path[-1] in prev:
        path.append(prev[path[-1]])
        if len(path) > MAX_PATH_LENGTH:
            break  # Enforce maximum path length
    return list(reversed(path))

# Alternative path finding using A* for shorter, more direct paths
def find_alt_path(start, target, max_length=MAX_PATH_LENGTH):
    """Find alternative path using A* search for more direct routes."""
    if start not in H or target not in H:
        return None
        
    # A* search with heuristic based on embedding similarity
    def heuristic(n):
        if n in emb and target in emb:
            return 1 - cosine(emb[n], emb[target])  # Lower for similar nodes
        return 1.0
    
    open_set = [(0, 0, start, [start])]  # (f_score, g_score, node, path)
    closed_set = set()
    
    while open_set:
        _, g_score, current, path = heapq.heappop(open_set)
        
        if current == target:
            return path
            
        if current in closed_set or len(path) > max_length:
            continue
            
        closed_set.add(current)
        
        for _, neighbor in H.out_edges(current):
            if neighbor in closed_set:
                continue
                
            tentative_g = g_score + 1
            f_score = tentative_g + heuristic(neighbor)
            
            heapq.heappush(open_set, (f_score, tentative_g, neighbor, path + [neighbor]))
    
    return None

# Find connections from user skills to orphans
def find_path_to_user_skill(target_id):
    """Try to find a path from any user skill to the target."""
    if not user_ids:
        return None
        
    # Try each user skill
    best_path = None
    best_sim = -1
    
    for user_id in user_ids:
        # Direct semantic similarity
        if target_id in emb and user_id in emb:
            sim = cosine(emb[user_id], emb[target_id])
            if sim > ORPHAN_SIM_THRESHOLD and sim > best_sim:
                best_sim = sim
                best_path = [user_id, target_id]
        
        # Try A* search for a path
        path = find_alt_path(user_id, target_id, max_length=3)
        if path and len(path) > 1:
            path_quality = sum(cosine(emb[path[i]], emb[path[i+1]]) 
                              for i in range(len(path)-1) 
                              if path[i] in emb and path[i+1] in emb) / (len(path)-1)
            if path_quality > best_sim:
                best_sim = path_quality
                best_path = path
    
    return best_path

# ─────────── USER INPUT ──────────────────────────────────
# ---------------- USER INPUT -----------------
user_skills = ["graph theory","probability","linear algebra"]
user_ids = { name2id[s.lower()] for s in user_skills if s.lower() in name2id }
# ---------------------------------------------

# helper: trim a path so it starts with the first node the user already knows
def trim_to_user(path):
    if path is None:
        return None
    for i, n in enumerate(path):
        if n in user_ids:
            return path[i:]
    return path  # Return the full path if no user skill found

# Rate the quality of a path based on relevance to job
def path_quality_score(path, job_emb):
    """Calculate a quality score for the path based on relevance to job."""
    if not path or len(path) <= 1 or job_emb is None:
        return 0.0
    
    # Average similarity to job
    job_sim = 0.0
    count = 0
    for node in path:
        if node in emb:
            job_sim += cosine(emb[node], job_emb)
            count += 1
    
    if count == 0:
        return 0.0
    
    # Penalize very long paths
    length_factor = max(0, 1 - 0.03 * (len(path) - 3))
    
    return (job_sim / count) * length_factor

# ─────────── 5. IMPROVED REPORT ──────────────
job_emb = emb.get(job_id)
reqs = set(edges.loc[edges[":START_ID"] == job_id, ":END_ID"].astype(int))
print(f"\n{JOB.upper()} — {len(reqs)} required skills\n")

# Group skills by category for better organization
skill_categories = {
    "already_known": [],      # Skills the user already has
    "ready_to_learn": [],     # Skills with direct paths from user knowledge
    "requires_prereqs": [],   # Skills requiring longer paths
    "connected_orphans": [],  # Orphans with alternative connections
    "true_orphans": []        # Truly disconnected skills
}

# Process each required skill
for sid in sorted(reqs, key=lambda x: id2name[x].lower()):
    if not (label_of[sid] & END_LABELS):
        continue  # ignore SoftSkills, Jobs, etc.
    
    skill_name = id2name[sid]
    
    # Check if already known
    if sid in user_ids:
        skill_categories["already_known"].append((sid, [sid]))
        continue
        
    # Try to find optimal path via DP
    dp_chain = build_chain(sid)
    dp_chain_anchored = trim_to_user(dp_chain) if dp_chain else None
    
    # If we have a good DP path anchored to user skills
    if dp_chain_anchored and len(dp_chain_anchored) > 1 and dp_chain_anchored[0] in user_ids:
        if len(dp_chain_anchored) <= 3:
            skill_categories["ready_to_learn"].append((sid, dp_chain_anchored))
        else:
            skill_categories["requires_prereqs"].append((sid, dp_chain_anchored))
        continue
    
    # If we have a DP path but not anchored to user skills
    if dp_chain and len(dp_chain) > 1:
        # Try to find a shorter alternative path from user skills
        alt_path = None
        for user_id in user_ids:
            user_to_first = find_alt_path(user_id, dp_chain[0], max_length=2)
            if user_to_first:
                alt_path = user_to_first[:-1] + dp_chain  # Connect without duplicating node
                break
        
        if alt_path:
            if len(alt_path) <= 4:
                skill_categories["ready_to_learn"].append((sid, alt_path))
            else:
                skill_categories["requires_prereqs"].append((sid, alt_path))
            continue
        else:
            skill_categories["requires_prereqs"].append((sid, dp_chain))
            continue
    
    # For orphans, try semantic connections to user skills
    user_path = find_path_to_user_skill(sid)
    if user_path:
        skill_categories["connected_orphans"].append((sid, user_path))
        continue
    
    # True orphan - look for dependent skills
    deps = edges.loc[edges[":END_ID"] == sid, ":START_ID"].astype(int)
    cands = [d for d in deps if (CONCEPT in label_of.get(d,()) or HARDSKILL in label_of.get(d,()))]
    
    scored = []
    for d in cands:
        r = edges[(edges[":START_ID"]==d)&(edges[":END_ID"]==sid)].iloc[0]
        sc = abs(float(r["score:float"])) + (PRED if r["predicted:boolean"] else 0.0)
        
        # Prefer concepts that have paths from user skills
        d_chain = build_chain(d)
        d_chain_anchored = trim_to_user(d_chain) if d_chain else None
        
        if d_chain_anchored and d_chain_anchored[0] in user_ids:
            sc *= 0.8  # Boost score (lower is better)
            
        scored.append((d, sc, d_chain_anchored or d_chain))
        
    scored.sort(key=lambda x: x[1])
    skill_categories["true_orphans"].append((sid, [c for c, _, path in scored[:SUGGEST_K]]))

# Print results by category
print("\n=== SKILLS YOU ALREADY KNOW ===")
for sid, _ in skill_categories["already_known"]:
    print(f"• {id2name[sid]}")

print("\n=== SKILLS READY TO LEARN ===")
for sid, path in skill_categories["ready_to_learn"]:
    print(f"• {id2name[sid]:30}  depth={len(path)-1}")
    print("    ", " → ".join(id2name[n] for n in path), "\n")

print("\n=== SKILLS REQUIRING PREREQUISITES ===")
for sid, path in skill_categories["requires_prereqs"]:
    print(f"• {id2name[sid]:30}  depth={len(path)-1}")
    print("    ", " → ".join(id2name[n] for n in path), "\n")

print("\n=== ORPHAN SKILLS WITH CONNECTIONS TO YOUR KNOWLEDGE ===")
for sid, path in skill_categories["connected_orphans"]:
    print(f"• {id2name[sid]:30}  (semantic connection)")
    print("    ", " → ".join(id2name[n] for n in path), "\n")

print("\n=== ORPHAN SKILLS ===")
for sid, dependents in skill_categories["true_orphans"]:
    print(f"✘ {id2name[sid]:30}  — no direct path from your skills")
    print("    Concepts that depend on this skill (learn next):")
    for d in dependents:
        alt = build_chain(d) or [d]
        anchored = trim_to_user(alt) if user_ids else None
        final_alt = anchored if anchored else alt
        print("    •", " → ".join(id2name[n] for n in final_alt))
    print()

Added 0 edges to fill structural gaps

AI ARCHITECT — 31 required skills


=== SKILLS YOU ALREADY KNOW ===

=== SKILLS READY TO LEARN ===

=== SKILLS REQUIRING PREREQUISITES ===
• artificial intelligence         depth=4
     radiance → specular surfaces → texture → feature extraction → artificial intelligence 

• computer science                depth=2
     foundations of mathematics → mathematics → computer science 

• google cloud platform (gcp)     depth=1
     gcp → google cloud platform (gcp) 

• scalability                     depth=5
     foundations of mathematics → mathematics → computer science → computer programming → software design → scalability 

• software development            depth=3
     foundations of mathematics → mathematics → computer science → software development 


=== ORPHAN SKILLS WITH CONNECTIONS TO YOUR KNOWLEDGE ===

=== ORPHAN SKILLS ===
✘ ai software                     — no direct path from your skills
    Concepts that depend on this skill (learn next

## Comparison


In [8]:
# ─────────────────────────────── COMMON CODE CELL ──────────────────────────────
import pandas as pd
import networkx as nx
import torch
import math
from pathlib import Path
from numpy.linalg import norm
import heapq
from collections import Counter

# ─── config ────────────────────────────────────────────────────────────────────
NODES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/nodes_final.csv")
EDGES_CSV   = Path("/teamspace/studios/this_studio/neo4j_export_csv/relationships_final.csv")
EMB_FILE    = Path("node_embeddings.pt")

JOB         = "nlp engineer"                # ← your job title
CONCEPT     = "Concept"
HARDSKILL   = "HardSkill"
TECH        = "Technology"
END_LABELS  = {HARDSKILL, TECH}              # only these count as end-skills
SUGGEST_K   = 5                              # how many concepts to suggest for orphans
MAX_PATH_LENGTH = 15                       # maximum reasonable path length

# Scoring parameters
Δ, α, β, PRED = 0.3, 0.9, 0.1, 0.7        # scoring constants
DOMAIN_THRESH = 0.7                       # semantic similarity threshold
SIM_BOOST = 0.6                            # boost for semantically similar nodes
LENGTH_PENALTY = 0.03                      # penalty for longer paths

# ─────────────────────────────────────────────────────────────────────────────

# 1) LOAD CSVs & EMBEDDINGS
nodes = pd.read_csv(NODES_CSV)
edges = pd.read_csv(EDGES_CSV)
emb   = torch.load(str(EMB_FILE), map_location="cpu")

# build label lookup
label_of = {
    int(r["nodeId:ID"]): set(r["labels:LABEL"].split(";"))
    for _, r in nodes.iterrows()
}

# safe name lookup
def name_of(n):
    row = nodes.loc[nodes["nodeId:ID"] == n].iloc[0]
    if "name" in row and pd.notna(row["name"]):
        return row["name"]
    if "title" in row and pd.notna(row["title"]):
        return row["title"]
    return str(n)

# id↔name maps
id2name = {nid: name_of(nid) for nid in label_of}
name2id = {n.lower(): nid for nid, n in id2name.items()}

job_id = name2id.get(JOB.lower())
if job_id is None:
    raise KeyError(f"Job '{JOB}' not found in nodes")
Smax = edges["score:float"].abs().max()

# cosine similarity
def cosine(u, v):
    return float((u @ v) / (norm(u) * norm(v) + 1e-9))

# 2) BUILD learning-DAG H: prereq → dependent
H = nx.DiGraph()
H.add_nodes_from(label_of.keys())

# Extract all edge information first to help identify important relationships
edge_counts = Counter()
for _, r in edges.iterrows():
    dep, pre = int(r[":START_ID"]), int(r[":END_ID"])  # CSV: dependent→prereq
    edge_counts[(dep, pre)] += 1

for _, r in edges.iterrows():
    dep, pre = int(r[":START_ID"]), int(r[":END_ID"])  # CSV: dependent→prereq
    labs_pre = label_of.get(pre, ())
    labs_dep = label_of.get(dep, ())

    # only keep if prereq is Concept or HardSkill, and dependent isn't Technology
    if not (CONCEPT in labs_pre or HARDSKILL in labs_pre):
        continue
    if TECH in labs_dep:
        continue

    raw  = abs(float(r["score:float"]))
    
    # Give small boost to frequently occurring edges - indicates importance
    importance_factor = min(1.0 + 0.05 * edge_counts[(dep, pre)], 1.3)
    raw = raw * importance_factor
    
    cost = raw + (PRED if r["predicted:boolean"] else 0.0)

    # Semantic similarity boost
    if pre in emb and dep in emb:
        sim = cosine(emb[pre], emb[dep])
        if sim > 0.7:  # High similarity
            cost = cost * (1.0 - SIM_BOOST)  # Lower cost for similar concepts

    # avoid cycles
    if nx.has_path(H, dep, pre):
        continue

    # insert or tighten existing edge
    if H.has_edge(pre, dep):
        H[pre][dep]["cost"] = min(H[pre][dep]["cost"], cost)
        H[pre][dep]["raw"]  = min(H[pre][dep]["raw"],  raw)
    else:
        H.add_edge(pre, dep, cost=cost, raw=raw)

assert nx.is_directed_acyclic_graph(H), "H must be a DAG!"

# Find structural gaps in the knowledge graph
def detect_and_fill_gaps():
    """
    Detect structural gaps in the graph by finding skills that should be connected
    based on semantic similarity but aren't linked in the graph.
    """
    tech_nodes = [n for n in H.nodes() if TECH in label_of.get(n, set())]
    skills_nodes = [n for n in H.nodes() if HARDSKILL in label_of.get(n, set())]
    concept_nodes = [n for n in H.nodes() if CONCEPT in label_of.get(n, set())]
    
    additions = 0
    
    # Look for technologies that should connect to relevant skills
    for tech in tech_nodes:
        if tech not in emb:
            continue
            
        # Find semantically similar skills or concepts
        similar_nodes = []
        for skill in skills_nodes + concept_nodes:
            if skill not in emb:
                continue
                
            sim = cosine(emb[tech], emb[skill])
            if sim > 0.65:  # High similarity threshold
                similar_nodes.append((skill, sim))
        
        # Sort by similarity and connect the top matches if not already connected
        similar_nodes.sort(key=lambda x: x[1], reverse=True)
        for skill, sim in similar_nodes[:3]:  # Top 3 most similar
            if not H.has_edge(skill, tech) and not nx.has_path(H, tech, skill):
                # Add edge from skill to tech (skill is prerequisite for tech)
                H.add_edge(skill, tech, cost=0.6, raw=0.6)
                additions += 1
    
    # Connect isolated skills to relevant concepts
    orphan_skills = [n for n in H.nodes() 
                    if HARDSKILL in label_of.get(n, set()) 
                    and H.in_degree(n) == 0
                    and H.out_degree(n) == 0]
    
    for orphan in orphan_skills:
        if orphan not in emb:
            continue
            
        # Find relevant concepts
        similar_concepts = []
        for concept in concept_nodes:
            if concept not in emb:
                continue
                
            sim = cosine(emb[orphan], emb[concept])
            if sim > 0.6:
                similar_concepts.append((concept, sim))
        
        # Connect to top matches
        similar_concepts.sort(key=lambda x: x[1], reverse=True)
        for concept, sim in similar_concepts[:2]:  # Top 2
            # Add bidirectional edges to represent soft relationships
            if not nx.has_path(H, orphan, concept) and not nx.has_path(H, concept, orphan):
                H.add_edge(concept, orphan, cost=0.7, raw=0.7)
                additions += 1
    
    print(f"Added {additions} edges to fill structural gaps")
    return additions

# Fill graph gaps
detect_and_fill_gaps()

# ─────────── USER INPUT ──────────────────────────────────
# ---------------- USER INPUT -----------------
user_skills = ["graph theory", "computer vision", "neural network", "reinforcement learning"]
user_ids = { name2id[s.lower()] for s in user_skills if s.lower() in name2id }
# ---------------------------------------------

# helper: trim a path so it starts with the first node the user already knows
def trim_to_user(path):
    if path is None:
        return None
    for i, n in enumerate(path):
        if n in user_ids:
            return path[i:]
    return path  # Return the full path if no user skill found

# Common function to get required skills for the job
def get_required_skills():
    reqs = set(edges.loc[edges[":START_ID"] == job_id, ":END_ID"].astype(int))
    return reqs

# Common function to categorize skills
def create_skill_categories():
    return {
        "already_known": [],      # Skills the user already has
        "ready_to_learn": [],     # Skills with direct paths from user knowledge
        "requires_prereqs": [],   # Skills requiring longer paths
        "connected_orphans": [],  # Orphans with alternative connections
        "true_orphans": []        # Truly disconnected skills
    }

# Common function to print the results
def print_skill_categories(skill_categories):
    print("\n=== SKILLS YOU ALREADY KNOW ===")
    for sid, _ in skill_categories["already_known"]:
        print(f"• {id2name[sid]}")

    print("\n=== SKILLS READY TO LEARN ===")
    for sid, path in skill_categories["ready_to_learn"]:
        print(f"• {id2name[sid]:30}  depth={len(path)-1}")
        print("    ", " → ".join(id2name[n] for n in path), "\n")

    print("\n=== SKILLS REQUIRING PREREQUISITES ===")
    for sid, path in skill_categories["requires_prereqs"]:
        print(f"• {id2name[sid]:30}  depth={len(path)-1}")
        print("    ", " → ".join(id2name[n] for n in path), "\n")

    print("\n=== ORPHAN SKILLS WITH CONNECTIONS TO YOUR KNOWLEDGE ===")
    for sid, path in skill_categories["connected_orphans"]:
        print(f"• {id2name[sid]:30}  (semantic connection)")
        print("    ", " → ".join(id2name[n] for n in path), "\n")

    print("\n=== ORPHAN SKILLS ===")
    for sid, dependents in skill_categories["true_orphans"]:
        print(f"✘ {id2name[sid]:30}  — no direct path from your skills")
        print("    Concepts that depend on this skill (learn next):")
        for d in dependents:
            alt = build_chain_dp(d) or [d]
            anchored = trim_to_user(alt) if user_ids else None
            final_alt = anchored if anchored else alt
            print("    •", " → ".join(id2name[n] for n in final_alt))
        print()

Added 0 edges to fill structural gaps


In [1]:
# ─────────────────────────────── DYNAMIC PROGRAMMING CELL ──────────────────────────────
# Run the common code cell first before executing this cell

# 3) DP: longest + best-score path to every node with improved path quality
dp = {n: -math.inf for n in H}
prev = {}
path_lens = {n: 0 for n in H}  # Keep track of path lengths

# seed roots: any Concept/HardSkill with zero prerequisites
for n in H:
    if H.in_degree(n) == 0 and (CONCEPT in label_of[n] or HARDSKILL in label_of[n]):
        dp[n] = 0.0
        path_lens[n] = 1

# Improved DP prioritizing path quality and reasonable length
for u in nx.topological_sort(H):
    if dp[u] < -1e8:
        continue
    for _, v, d in H.out_edges(u, data=True):
        s_norm = d["raw"] / Smax if Smax else 0.0
        sem_pen = (1 - cosine(emb[u], emb[v])) if (u in emb and v in emb) else 1.0
        
        # Add length penalty to discourage excessively long paths
        length_factor = LENGTH_PENALTY * path_lens[u]
        
        gain = Δ - α * s_norm - β * sem_pen - length_factor
        cand = dp[u] + gain
        
        # Only consider if path length is reasonable
        new_path_len = path_lens[u] + 1
        if new_path_len <= MAX_PATH_LENGTH and cand > dp[v]:
            dp[v], prev[v] = cand, u
            path_lens[v] = new_path_len

# back-track chain builder with length consideration
def build_chain_dp(n):
    if dp.get(n, -math.inf) < -1e8:
        return None
    path = [n]
    while path[-1] in prev:
        path.append(prev[path[-1]])
        if len(path) > MAX_PATH_LENGTH:
            break  # Enforce maximum path length
    return list(reversed(path))

# Function to find orphan skills' dependents (for skills without paths)
def find_orphan_dependents(sid):
    deps = edges.loc[edges[":END_ID"] == sid, ":START_ID"].astype(int)
    cands = [d for d in deps if (CONCEPT in label_of.get(d,()) or HARDSKILL in label_of.get(d,()))]
    
    scored = []
    for d in cands:
        r = edges[(edges[":START_ID"]==d)&(edges[":END_ID"]==sid)].iloc[0]
        sc = abs(float(r["score:float"])) + (PRED if r["predicted:boolean"] else 0.0)
        scored.append((d, sc))
    scored.sort(key=lambda x: x[1])
    
    return [d for d, _ in scored[:SUGGEST_K]]

# Run the DP-based analysis on required skills
def run_dp_analysis():
    print(f"\n{JOB.upper()} — DYNAMIC PROGRAMMING ANALYSIS\n")
    
    reqs = get_required_skills()
    skill_categories = create_skill_categories()
    
    # Process each required skill
    for sid in sorted(reqs, key=lambda x: id2name[x].lower()):
        if not (label_of[sid] & END_LABELS):
            continue  # ignore SoftSkills, Jobs, etc.
        
        # Check if already known
        if sid in user_ids:
            skill_categories["already_known"].append((sid, [sid]))
            continue
            
        # Try to find path via DP
        dp_chain = build_chain_dp(sid)
        dp_chain_anchored = trim_to_user(dp_chain) if dp_chain else None
        
        # If we have a good DP path anchored to user skills
        if dp_chain_anchored and len(dp_chain_anchored) > 1 and dp_chain_anchored[0] in user_ids:
            if len(dp_chain_anchored) <= 3:
                skill_categories["ready_to_learn"].append((sid, dp_chain_anchored))
            else:
                skill_categories["requires_prereqs"].append((sid, dp_chain_anchored))
            continue
        
        # If we have a DP path but not anchored to user skills
        if dp_chain and len(dp_chain) > 1:
            skill_categories["requires_prereqs"].append((sid, dp_chain))
            continue
        
        # Handle orphan skills - look for dependent skills
        dependents = find_orphan_dependents(sid)
        skill_categories["true_orphans"].append((sid, dependents))
    
    # Print results
    print_skill_categories(skill_categories)
    
    return skill_categories

# Run the analysis
dp_skill_categories = run_dp_analysis()

NameError: name 'H' is not defined

In [10]:
# ─────────────────────────────── A* SEARCH CELL ──────────────────────────────
# Run the common code cell first before executing this cell

# A* search implementation for finding paths
def find_astar_path(start, target, max_length=MAX_PATH_LENGTH):
    """Find path using A* search with embedding similarity as heuristic."""
    if start not in H or target not in H:
        return None
        
    # A* search with heuristic based on embedding similarity
    def heuristic(n):
        if n in emb and target in emb:
            return 1 - cosine(emb[n], emb[target])  # Lower for similar nodes
        return 1.0
    
    open_set = [(0, 0, start, [start])]  # (f_score, g_score, node, path)
    closed_set = set()
    
    while open_set:
        _, g_score, current, path = heapq.heappop(open_set)
        
        if current == target:
            return path
            
        if current in closed_set or len(path) > max_length:
            continue
            
        closed_set.add(current)
        
        for _, neighbor in H.out_edges(current):
            if neighbor in closed_set:
                continue
                
            tentative_g = g_score + 1
            f_score = tentative_g + heuristic(neighbor)
            
            heapq.heappush(open_set, (f_score, tentative_g, neighbor, path + [neighbor]))
    
    return None

# Find paths from any user skill to a target skill
def find_paths_from_user_skills(target_id, max_length=MAX_PATH_LENGTH):
    """Find paths from any user skill to the target skill."""
    if not user_ids:
        return None
    
    best_path = None
    best_score = float('inf')  # Lower is better for A*
    
    for user_id in user_ids:
        path = find_astar_path(user_id, target_id, max_length)
        if path and (best_path is None or len(path) < best_score):
            best_path = path
            best_score = len(path)
    
    return best_path

# Find semantic connections between skills
def find_semantic_connection(target_id, threshold=0.5):
    """Find semantically similar user skills."""
    if not user_ids or target_id not in emb:
        return None
    
    best_match = None
    best_sim = -1
    
    for user_id in user_ids:
        if user_id not in emb:
            continue
        
        sim = cosine(emb[user_id], emb[target_id])
        if sim > threshold and sim > best_sim:
            best_sim = sim
            best_match = user_id
    
    if best_match:
        return [best_match, target_id]
    return None

# Function to find orphan skills' dependents using A*
def find_orphan_paths_astar(sid):
    deps = edges.loc[edges[":END_ID"] == sid, ":START_ID"].astype(int)
    cands = [d for d in deps if (CONCEPT in label_of.get(d,()) or HARDSKILL in label_of.get(d,()))]
    
    paths = []
    for d in cands[:SUGGEST_K]:  # Only process top K dependents
        # Try to find paths from user skills to this dependent
        best_path = None
        for user_id in user_ids:
            path = find_astar_path(user_id, d, max_length=3)
            if path:
                if not best_path or len(path) < len(best_path):
                    best_path = path
        
        if best_path:
            paths.append((d, best_path))
        else:
            paths.append((d, [d]))  # Just the node itself
    
    # Sort paths by length (shorter is better)
    paths.sort(key=lambda x: len(x[1]))
    return [d for d, _ in paths[:SUGGEST_K]]

# Run the A* analysis on required skills
def run_astar_analysis():
    print(f"\n{JOB.upper()} — A* SEARCH ANALYSIS\n")
    
    reqs = get_required_skills()
    skill_categories = create_skill_categories()
    
    # Process each required skill
    for sid in sorted(reqs, key=lambda x: id2name[x].lower()):
        if not (label_of[sid] & END_LABELS):
            continue  # ignore SoftSkills, Jobs, etc.
        
        # Check if already known
        if sid in user_ids:
            skill_categories["already_known"].append((sid, [sid]))
            continue
            
        # Try to find direct A* path from user skills
        astar_path = find_paths_from_user_skills(sid, max_length=MAX_PATH_LENGTH)
        
        if astar_path and len(astar_path) > 1:
            if len(astar_path) <= 3:
                skill_categories["ready_to_learn"].append((sid, astar_path))
            else:
                skill_categories["requires_prereqs"].append((sid, astar_path))
            continue
        
        # Try semantic connections for orphans
        semantic_path = find_semantic_connection(sid, threshold=0.45)
        if semantic_path:
            skill_categories["connected_orphans"].append((sid, semantic_path))
            continue
        
        # Handle true orphan skills
        dependents = find_orphan_paths_astar(sid)
        skill_categories["true_orphans"].append((sid, dependents))
    
    # Print results
    print_skill_categories(skill_categories)
    
    return skill_categories

# Run the analysis
astar_skill_categories = run_astar_analysis()


NLP ENGINEER — A* SEARCH ANALYSIS


=== SKILLS YOU ALREADY KNOW ===

=== SKILLS READY TO LEARN ===
• computer science                depth=1
     graph theory → computer science 

• natural language processing     depth=1
     graph theory → natural language processing 

• software engineering            depth=2
     graph theory → computer science → software engineering 


=== SKILLS REQUIRING PREREQUISITES ===
• scalability                     depth=4
     graph theory → computer science → programming language → software design → scalability 


=== ORPHAN SKILLS WITH CONNECTIONS TO YOUR KNOWLEDGE ===

=== ORPHAN SKILLS ===
✘ algorithms                      — no direct path from your skills
    Concepts that depend on this skill (learn next):
    • foundations of mathematics → mathematics → probability → context sensitive grammar → anaphora resolution → discourse model
    • cuda → message passing interface
    • algorithms → address locator
    • genai
    • graph theory → computer 

In [7]:
# ─────────────────────────────── COMBINED APPROACH CELL ──────────────────────────────
# Run the common code cell first, then run both DP and A* cells before executing this cell

# Combined path-finding approach that leverages both DP and A*
def find_best_path(sid):
    """Find the best path to a skill using both DP and A*."""
    best_path = None
    best_score = float('inf')  # Lower is better
    
    # 1. Try DP path
    dp_path = build_chain_dp(sid)
    dp_path_anchored = trim_to_user(dp_path) if dp_path else None
    
    if dp_path_anchored and dp_path_anchored[0] in user_ids:
        best_path = dp_path_anchored
        best_score = len(dp_path_anchored)
    elif dp_path:
        best_path = dp_path
        best_score = len(dp_path) + 10  # Penalty for not being anchored
    
    # 2. Try A* paths from each user skill
    for user_id in user_ids:
        astar_path = find_astar_path(user_id, sid, max_length=MAX_PATH_LENGTH)
        if astar_path and len(astar_path) < best_score:
            best_path = astar_path
            best_score = len(astar_path)
    
    # 3. If we still don't have a path, try connecting through intermediates
    if best_path is None and dp_path:
        # Try to connect user skills to the start of the DP path
        for user_id in user_ids:
            bridge_path = find_astar_path(user_id, dp_path[0], max_length=2)
            if bridge_path:
                # Combine paths without duplicating the connecting node
                combined_path = bridge_path[:-1] + dp_path
                if best_path is None or len(combined_path) < best_score:
                    best_path = combined_path
                    best_score = len(combined_path)
    
    return best_path

# Find semantic connections for true orphans
def find_best_semantic_connection(sid, threshold=0.4):
    """Find the best semantic connection to user skills."""
    if sid not in emb:
        return None
    
    # Direct semantic similarity
    best_direct = find_semantic_connection(sid, threshold)
    
    # Two-hop semantic bridges
    best_bridge = None
    best_bridge_score = -1
    
    # Find intermediates that connect user skills to the target
    for n in H.nodes():
        if n not in emb or n == sid:
            continue
        
        # Check if this node connects well to both user skills and target
        target_sim = cosine(emb[n], emb[sid])
        if target_sim < 0.5:  # Minimum similarity to target
            continue
        
        # Check similarity to user skills
        for user_id in user_ids:
            if user_id not in emb:
                continue
            
            user_sim = cosine(emb[user_id], emb[n])
            if user_sim > 0.5:  # Good similarity to user skill
                bridge_score = (user_sim + target_sim) / 2
                if bridge_score > best_bridge_score:
                    best_bridge_score = bridge_score
                    best_bridge = [user_id, n, sid]
    
    # Return the best option
    if best_bridge and best_bridge_score > 0.6:
        return best_bridge
    return best_direct

# Improved orphan handling
def handle_orphan_skill(sid):
    """Comprehensive approach for orphan skills."""
    # 1. Try semantic connections
    semantic_path = find_best_semantic_connection(sid)
    if semantic_path:
        return "connected_orphans", semantic_path
    
    # 2. Find dependents
    deps = edges.loc[edges[":END_ID"] == sid, ":START_ID"].astype(int)
    cands = [d for d in deps if (CONCEPT in label_of.get(d,()) or HARDSKILL in label_of.get(d,()))]
    
    # 3. Score dependents by multiple factors
    scored_deps = []
    for d in cands:
        # Get edge score
        r = edges[(edges[":START_ID"]==d)&(edges[":END_ID"]==sid)].iloc[0]
        raw_score = abs(float(r["score:float"])) + (PRED if r["predicted:boolean"] else 0.0)
        
        # Check if this dependent has a path from user skills
        path_quality = 0
        best_path = find_best_path(d)
        if best_path and best_path[0] in user_ids:
            path_quality = 1.0 / len(best_path)  # Shorter paths are better
        
        # Combine factors
        final_score = raw_score * (1 - 0.3 * path_quality)  # Lower is better
        scored_deps.append((d, final_score, best_path))
    
    # Sort by final score
    scored_deps.sort(key=lambda x: x[1])
    
    # Return top dependents
    return "true_orphans", [d for d, _, _ in scored_deps[:SUGGEST_K]]

# Run the combined analysis on required skills
def run_combined_analysis():
    print(f"\n{JOB.upper()} — COMBINED ANALYSIS (DP + A*)\n")
    
    reqs = get_required_skills()
    skill_categories = create_skill_categories()
    
    # Process each required skill
    for sid in sorted(reqs, key=lambda x: id2name[x].lower()):
        if not (label_of[sid] & END_LABELS):
            continue  # ignore SoftSkills, Jobs, etc.
        
        # Check if already known
        if sid in user_ids:
            skill_categories["already_known"].append((sid, [sid]))
            continue
            
        # Try to find the best path using both DP and A*
        best_path = find_best_path(sid)
        
        if best_path and len(best_path) > 1:
            # Check if anchored to user skills
            if best_path[0] in user_ids:
                if len(best_path) <= 3:
                    skill_categories["ready_to_learn"].append((sid, best_path))
                else:
                    skill_categories["requires_prereqs"].append((sid, best_path))
            else:
                skill_categories["requires_prereqs"].append((sid, best_path))
            continue
        
        # Handle orphan skills
        category, result = handle_orphan_skill(sid)
        skill_categories[category].append((sid, result))
    
    # Print results
    print_skill_categories(skill_categories)
    
    return skill_categories

# Run the combined analysis
combined_skill_categories = run_combined_analysis()

# Compare results from different approaches
def count_by_category(categories):
    return {k: len(v) for k, v in categories.items()}

print("\n=== COMPARISON OF APPROACHES ===")
print("Dynamic Programming:")
dp_counts = count_by_category(dp_skill_categories)
print(dp_counts)

print("\nA* Search:")
astar_counts = count_by_category(astar_skill_categories)
print(astar_counts)

print("\nCombined Approach:")
combined_counts = count_by_category(combined_skill_categories)
print(combined_counts)

# Calculate path lengths for comparison
def calculate_average_path_length(categories):
    path_lengths = []
    for category in ["ready_to_learn", "requires_prereqs", "connected_orphans"]:
        for _, path in categories[category]:
            if isinstance(path, list) and len(path) > 1:
                path_lengths.append(len(path))
    
    if not path_lengths:
        return 0
    return sum(path_lengths) / len(path_lengths)

print("\n=== AVERAGE PATH LENGTHS ===")
print(f"Dynamic Programming: {calculate_average_path_length(dp_skill_categories):.2f}")
print(f"A* Search: {calculate_average_path_length(astar_skill_categories):.2f}")
print(f"Combined Approach: {calculate_average_path_length(combined_skill_categories):.2f}")

# Calculate overall effectiveness (% of skills with valid paths)
def calculate_effectiveness(categories):
    total_skills = sum(len(v) for v in categories.values())
    orphan_count = len(categories["true_orphans"])
    
    if total_skills == 0:
        return 0
    return 100 * (total_skills - orphan_count) / total_skills

print("\n=== OVERALL EFFECTIVENESS (% of skills with valid paths) ===")
print(f"Dynamic Programming: {calculate_effectiveness(dp_skill_categories):.2f}%")
print(f"A* Search: {calculate_effectiveness(astar_skill_categories):.2f}%")
print(f"Combined Approach: {calculate_effectiveness(combined_skill_categories):.2f}%")


COMPUTER VISION ENGINEER — COMBINED ANALYSIS (DP + A*)


=== SKILLS YOU ALREADY KNOW ===
• computer vision

=== SKILLS READY TO LEARN ===
• computer science                depth=1
     graph theory → computer science 

• machine learning                depth=1
     graph theory → machine learning 

• object detection                depth=1
     computer vision → object detection 

• robotics                        depth=2
     graph theory → artificial intelligence → robotics 


=== SKILLS REQUIRING PREREQUISITES ===
• image processing                depth=3
     graph theory → artificial intelligence → domain adaptation → image processing 


=== ORPHAN SKILLS WITH CONNECTIONS TO YOUR KNOWLEDGE ===

=== ORPHAN SKILLS ===
✘ algorithms                      — no direct path from your skills
    Concepts that depend on this skill (learn next):
    • genai
    • cuda → message passing interface
    • foundations of mathematics → mathematics → polynomial → algorithm → distributed algorithms